In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [55]:
matrix = np.load('..\data\LakeErie\\analysis\matrix_noaa_nan.npy')
# calculate the correlation matrix
from scipy.stats import pearsonr
def pearson_test(matrix, i, j, k):
    data1 = matrix[:,i,k]
    data2 = matrix[:,j,k]
    index = np.logical_and(~np.isnan(data1), ~np.isnan(data2))
    data1 = data1[index]
    data2 = data2[index]
    corr, p = pearsonr(data1, data2)
    return corr, p
corrs = np.zeros((matrix.shape[1], matrix.shape[1], matrix.shape[2]))
ps = np.zeros((matrix.shape[1], matrix.shape[1], matrix.shape[2]))
for i in range(matrix.shape[1]):
    for j in range(matrix.shape[1]):
        for k in range(matrix.shape[2]):
            corr, p = pearson_test(matrix, i, j, k)
            corrs[i,j,k] = corr
            ps[i,j,k] = p

In [56]:
# Spatio Then Temporal (STT) 
from scipy.interpolate import interp1d
def fill_missing(matrix, corr, threshold=0.5):
# fill missing values with Spatio correlation
    matrix_filled = matrix.copy()
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            for k in range(matrix.shape[2]):
                if np.isnan(matrix[i][j][k]):
                    # find the correlations > threshold
                    index = np.logical_and(~np.isnan(matrix[i,:,k]), corr[j,:,k]>threshold)
                    if np.sum(index)>0:
                        # Weighted average + random noise
                        mean = np.sum(matrix[i,index,k]*corr[j,index,k])/np.sum(corr[j,index,k])
                        std = np.std(matrix[i,index,k])
                        n = np.sum(index)
                        matrix_filled[i,j,k] = mean + np.random.normal(0,std/n)
                    else:
                        continue
    # fill the rest missing values with temporal KNN interpolation
    for j in range(matrix.shape[1]):
        for k in range(matrix.shape[2]):
            x = np.arange(matrix.shape[0])
            y = matrix_filled[:,j,k]
            f = interp1d(x[~np.isnan(y)], y[~np.isnan(y)], kind='linear', fill_value='extrapolate')
            xnew = x[np.isnan(y)]
            ynew = f(xnew)
            matrix_filled[np.isnan(y),j,k] = ynew
    return matrix_filled

In [57]:
# fill missing values with STT algorithm
matrix_filled = fill_missing(matrix, corrs, threshold=0.25)

In [42]:
#PreDay
matrix_filled_preday = matrix.copy()
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        if np.isnan(matrix[i,j,0]):
            if i == 0:
                matrix_filled_preday[i,j,0] = np.nanmean(matrix[:,j,0] )
            else:
                matrix_filled_preday[i,j,0] = matrix_filled_preday[i-1,j,0]

In [48]:
#Mean
matrix_filled_mean = matrix.copy()
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        if np.isnan(matrix[i,j,0]):
            matrix_filled_mean[i,j,0] = np.nanmean(matrix[:,j,0] )
